# DISTRIBUTED HBV MODEL 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

#Working with masked array
import numpy.ma as ma
import math
import os
import glob

In [ ]:
import xarray as xr
import rasterio
import rioxarray
#Projecting the files
import cartopy.crs as ccrs  # projection

import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon

In [ ]:
#Library to compute flow directions
import datetime
from datetime import date
from datetime import timedelta

In [ ]:
# Cliping netcdf file

from shapely.geometry import mapping
import geopandas as gpd  #for reading the shapefile
import regionmask  #For masking dataset

In [ ]:
#sub catchment discharge
def qsim(qsim_ds,time):
    q_arr= qsim_ds.to_numpy()
    q_arr2= ma.masked_invalid(q_arr)
    
    qsum= np.sum(q_arr2,axis=(1,2))
    #df_q= pd.DataFrame(qsum)
    
    df_q= pd.DataFrame(qsum, index=time)
    df_q.rename(columns={0:'Qsim'}, inplace=True)
    
    df_q= df_q.rename_axis('Date')

    #df_q.index= time
    
    return df_q

In [ ]:
def maxbas_weights(maxbas):
    mb = int(np.ceil(maxbas))
    w = np.zeros(mb)
    t = np.arange(1, mb+1)
    for i in range(mb):
        if t[i] < maxbas / 2:
            w[i] = t[i] / (maxbas / 2)
        else:
            w[i] = (maxbas - t[i] + 1) / (maxbas / 2)
    w /= w.sum()
    return w

def apply_maxbas_routing(runoff, maxbas):
    weights = maxbas_weights(maxbas)
    routed = np.convolve(runoff, weights, mode='full')[:len(runoff)]
    return routed

def qsim_routed(qsim_ds, time, maxbas):
    """
    maxbas: parameter for triangular weighting/routing
    """
    q_arr = qsim_ds.to_numpy()
    q_arr2 = ma.masked_invalid(q_arr)
    qsum = np.sum(q_arr2, axis=(1,2))
    
    # Apply maxbas routing (HBV triangular convolution)
    qsum_routed = apply_maxbas_routing(qsum, maxbas)
    
    df_q = pd.DataFrame(qsum_routed, index=time)
    df_q.rename(columns={0:'Qsim'}, inplace=True)
    df_q = df_q.rename_axis('Date')
    
    return df_q

In [ ]:
df_qobs= pd.read_excel(r"Discharge series.xlsx", index_col=0,parse_dates= True)
df_qobs.head()

# A.1 Subcatchments

# Reading shape files

In [ ]:
shp1= gpd.read_file(r"subbasin1.shp")
shp2= gpd.read_file(r"subbasin2.shp")
shp3= gpd.read_file(r"subbasin3.shp")
shp4= gpd.read_file(r"subbasin4.shp")

# Creating a function to define a subbasin dem:
def subbasin_clip(dem_cat, shp):
    dem_sub= raster.rio.clip(shp.geometry.apply(mapping), shp.crs, all_touched= True, drop= True)
    return dem_sub

In [ ]:
def clip2(cm, shp):
    cm = cm.rename({'lat':'y', 'lon':'x'}) # Specifiying the coordinate system
    cm.rio.write_crs("EPSG:3067",inplace= True)
    
    #cm_fin= cm.rio.clip(shp.geometry.apply(mapping), shp.crs, all_touched= True, drop= True)
    cm_fin= cm.rio.clip(shp.geometry.apply(mapping), shp.crs)
    
    cm_fin= cm_fin.rename({'y':'lat', 'x':'lon'})
    return cm_fin

# B. HBV ROUTINE

### Climate forcing Data

In [ ]:
#Input data
ds_temp= xr.open_dataset(r"DailyTemp.nc")
ds_prec = xr.open_dataset(r"DailyPrec.nc")
ds_evap= xr.open_dataset(r"DailyEvap.nc")

In [ ]:

ds= ds_prec
#Creating arrays
prec= np.array(ds_prec.pr)
temp= np.array(ds_temp.tas)
evap= np.array(ds_evap.ET0) 
 

## B.1 HBV Routine computations

### Abbreviation definitions

- PCORR     Precipitation correction (usually around 1.05) 
- SCORR     Snow correction (usually around 1.2) 
- TCGRAD    Temperature lapse rate dry day (degree/100 m) 
- TPGRAD    Temperature lapse rate wet day (degree/100 m) 
- PGRAD      Precipitation lapse rate (%/100 m) 

#### Snow routine States and Parameters 
- SN     Dry snow (mm) 
- SW     Water content in snow (mm)
- SWE    Snow water equivalent


- SMLT    Computed snow melt (mm) 
- SR      Computed refrozen water (mm) 
- ST      Computed free water limit in the snow pack (mm) 
- INSOIL  Computed water going to the soil routine (mm) 


- TX     Transition temperature snow – rain (deg.C) 
- TS     Boundary temperature for snowmelt (deg.C) 
- CX     Degree day factor (mm/deg.C * day) 
- CFR    Refreeze factor (mm/deg.C * day) 
- CPRO   Liquid water content in snow (%) 
- CXN    Degree day factor in forested areas (PINEHBV) 
- TSN    Boundary temperature in forested areas (PINEHBV)

#### Soil routine States and Parameters

- SM    Soil moisture content (mm) 

- EA     Actual evaporation (mm) 
- dUZ    Water to upper zone (mm)  

- FC    Field capacity (mm) 
- Beta  Exponent in function defining storage and output (-) 
- LP    Boundary for full evaporation (0-1) 

- UZ    Water content in upper zone (mm) 
- Q11   Runoff from upper outlet (mm) 
- Q10   Runoff from lower outlet (mm) 

#### Upper and Lower States and Parameters

- KUZ1  Upper outlet constant (-) 
- KUZ  Lower outlet constant (-) 
- UZ1  Treshold for activation of upper outlet (mm) 
- PERC  Percolation, water transport from upper to lower zone (mm) 

- Note that KUZ1 is always lager than KUZ.


In [ ]:
def hbv_model(params,params2, prec, temp, evap, ds_oul):
    # Model parameters: [Pcorr, Scorr, Tx, Ts, Cx  FC, Beta, LP, K1, K2,UZ1, KLZ, PERC]
    Pcorr, Scorr, Tx, Ts, CX, FC, Beta, LP, K1, K2, UZ1, KLZ, PERC = params
    
    #Free parameters
    Pgrad, Tgrad, Area, CFR, CFRO, LA= params2

    #creating empty array for the different states 
    #Defining the snow routine

    SN=  np.zeros_like(prec) #Dry snow pack
    Prain= np.zeros_like(prec) # Precipitation as Rain
    Psnow= np.zeros_like(prec) # Precipitation as snow
    SMLT= np.zeros_like(prec) #Snow melt 
    SR= np.zeros_like(prec) #Refreeze
    ST=  np.zeros_like(prec)#Free water in the snow pack
    INSOIL= np.zeros_like(prec) # To the soil routine
    SW= np.zeros_like(prec) # Liquid water in snow

    #Defining the soil routine 
    dUZ= np.zeros_like(prec)
    EA = np.zeros_like(prec)
    SM= np.zeros_like(prec)


    #Defining the upper tank and lower tank routines
    #Upper tank
    UZ= np.zeros_like(prec)
    Q10 = np.zeros_like(prec)
    Q11= np.zeros_like(prec)

    LZ= np.zeros_like(prec)
    QLZ = np.zeros_like(prec)
    Qsim= np.zeros_like(prec)
    Qin= np.zeros_like(prec)
    Qdir = np.zeros_like(prec)



    #Defining initial states
    SN[0] = 17  # initial value for the simulated snow pack # Initial snow state
    SW[0] = 17  # inital liquid water in the snow pack

    SM[0]=  20.0 # initial soil moisture
    UZ[0] = 20.0 # Initial upper storage
    LZ[0] = 10.0 # Initial lower storage


    ## HBV Vertical routine


    #LA= lake_arr.copy()
    for i in range(1, len(prec)):
        #Getting rain and snow precipitation
        Prain[i]= np.where(temp[i]>Tx, prec[i]*Pcorr,0)
        Psnow[i]= np.where(temp[i]<=Tx, prec[i]*Scorr,0)

        #Snow routine
        # Snow melt equation 
        SMLT[i]= np.minimum(SN[i-1], np.maximum(0, CX*(temp[i]-Ts)))  #to ensure we dont melt more snow that available and also avoid negative melt

        # Snow refreeze
        SR[i]= np.minimum(SW[i-1], np.maximum(0,-1*CFR*CX*(temp[i]-Ts))) #Refreeze doesnt exceed the liquid water in the snow pack

        ST[i]= (SN[i-1]+Psnow[i]) *CFRO  # Computed free water limit in the snow pack

        #computing new states for the model
        SN[i] = SN[i-1]+ Psnow[i]-SMLT[i] + SR[i]  # Dry snow
        SW[i]= np.minimum(ST[i], SW[i-1]+ Prain[i]+SMLT[i]-SR[i])# Water content in snow


        #In Soil
        #All excess water from the snow is sent to the soil routine
        Qin[i]= np.maximum(0, (SW[i-1]+ Prain[i]+SMLT[i]-SR[i])-ST[i])

        #Direct runoff if the infilttration of soil is exceeded
        Qdir[i] = np.maximum(0, SM[i-1]+Qin[i]-FC)
        INSOIL[i]= Qin[i]-Qdir[i]

        #Soil routine 
        dUZ[i]= INSOIL[i]*(SM[i-1]/FC)**Beta  # water to the upper zone

        EA[i]= evap[i]*np.minimum(1, SM[i-1]/(LP*FC)) # Actual Evapouration
        SM[i] = SM[i-1]+ INSOIL[i]-dUZ[i]-EA[i] # Soil water content


        #Upper Zone routine

        Q10[i]= np.minimum((UZ[i-1]+ dUZ[i]+Qdir[i]-np.minimum(PERC,UZ[i-1])), UZ1)*K1

        Q11[i]= np.maximum(0, (UZ[i-1]+ dUZ[i]+Qdir[i]-np.minimum(PERC,UZ[i-1]))-UZ1)*K2
        UZ[i] = UZ[i-1]+ dUZ[i]+Qdir[i]-np.minimum(PERC,UZ[i-1])- Q10[i]-Q11[i]

        #Lower zone routine
        QLZ[i] = KLZ*(LZ[i-1]+np.minimum(PERC,UZ[i-1]))+ LA*(prec[i]-evap[i])*KLZ
        LZ[i]= np.maximum(0, (LZ[i-1]+ np.minimum(PERC,UZ[i-1])- QLZ[i] + LA*(prec[i]-evap[i])))


        #simulated discharge

        Qsim[i]= (Q10[i]+Q11[i]+ QLZ[i])*(Area*10**3)/(86400)


    # Extracting coordinates 
    lat= ds['lat'].to_numpy()
    lon= ds['lon'].to_numpy()
    time= ds['time'].to_numpy()


    #Converting a numpy array to xarray
    Qsim_dr= xr.DataArray(Qsim,
                          coords={'time':time, 'lat':lat, 'lon':lon},
                         dims=['time','lat', 'lon'])

    
    # to dataset
    Qsim_ds= Qsim_dr.to_dataset(name='Q', promote_attrs=True)


    #Qsim_dr= Qsim_ds.Q
    # Map basin names to their shapefiles (in order)
    basins = [
        ("subbasin1", shp1),
        ("subbasin2", shp2),
        ("subbasin3", shp3),
        ("subbasin4", shp4),
        ]

    # Build a dict of DataFrames: {name: qsim(clip2(Qsim_dr, shp))}
    df_dict = {name: qsim(clip2(Qsim_dr, shp),time) for name, shp in basins}

    # Concatenate along columns; collapse the second level if present
    df_simq = pd.concat(df_dict, axis=1)

    # Flatten MultiIndex columns if necessary
    df_simq.columns = df_simq.columns.droplevel(1)
    
    return df_simq


### HBV model with MAXBAS Routing 

In [ ]:
def hbv_model_maxbas(params,params2, prec, temp, evap, ds_oul):
    # Model parameters: [Pcorr, Scorr, Tx, Ts, Cx  FC, Beta, LP, K1, K2,UZ1, KLZ, PERC]
    Pcorr, Scorr, Tx, Ts, CX, FC, Beta, LP, K1, K2, UZ1, KLZ, PERC = params
    
    #Free parameters
    Pgrad, Tgrad, Area, CFR, CFRO, LA= params2

    #creating empty array for the different states 
    #Defining the snow routine

    SN=  np.zeros_like(prec) #Dry snow pack
    Prain= np.zeros_like(prec) # Precipitation as Rain
    Psnow= np.zeros_like(prec) # Precipitation as snow
    SMLT= np.zeros_like(prec) #Snow melt 
    SR= np.zeros_like(prec) #Refreeze
    ST=  np.zeros_like(prec)#Free water in the snow pack
    INSOIL= np.zeros_like(prec) # To the soil routine
    SW= np.zeros_like(prec) # Liquid water in snow

    #Defining the soil routine 
    dUZ= np.zeros_like(prec)
    EA = np.zeros_like(prec)
    SM= np.zeros_like(prec)


    #Defining the upper tank and lower tank routines
    #Upper tank
    UZ= np.zeros_like(prec)
    Q10 = np.zeros_like(prec)
    Q11= np.zeros_like(prec)

    LZ= np.zeros_like(prec)
    QLZ = np.zeros_like(prec)
    Qsim= np.zeros_like(prec)
    Qin= np.zeros_like(prec)
    Qdir = np.zeros_like(prec)



    #Defining initial states
    SN[0] = 17  # initial value for the simulated snow pack # Initial snow state
    SW[0] = 17  # inital liquid water in the snow pack

    SM[0]=  20.0 # initial soil moisture
    UZ[0] = 20.0 # Initial upper storage
    LZ[0] = 10.0 # Initial lower storage


    ## HBV Vertical routine


    #LA= lake_arr.copy()
    for i in range(1, len(prec)):
        #np.where is used to compute multiple conditions

        #Getting rain and snow precipitation
        Prain[i]= np.where(temp[i]>Tx, prec[i]*Pcorr,0)
        Psnow[i]= np.where(temp[i]<=Tx, prec[i]*Scorr,0)

        #Snow routine
        # Snow melt equation 
        SMLT[i]= np.minimum(SN[i-1], np.maximum(0, CX*(temp[i]-Ts)))  #to ensure we dont melt more snow that available and also avoid negative melt

        # Snow refreeze
        SR[i]= np.minimum(SW[i-1], np.maximum(0,-1*CFR*CX*(temp[i]-Ts))) #Refreeze doesnt exceed the liquid water in the snow pack

        ST[i]= (SN[i-1]+Psnow[i]) *CFRO  # Computed free water limit in the snow pack

        #computing new states for the model
        SN[i] = SN[i-1]+ Psnow[i]-SMLT[i] + SR[i]  # Dry snow
        SW[i]= np.minimum(ST[i], SW[i-1]+ Prain[i]+SMLT[i]-SR[i])# Water content in snow


        #In Soil
         Qin[i]= np.maximum(0, (SW[i-1]+ Prain[i]+SMLT[i]-SR[i])-ST[i])

        #Direct runoff if the infilttration of soil is exceeded
        Qdir[i] = np.maximum(0, SM[i-1]+Qin[i]-FC)
        INSOIL[i]= Qin[i]-Qdir[i]

        #Soil routine 
        dUZ[i]= INSOIL[i]*(SM[i-1]/FC)**Beta  # water to the upper zone

        EA[i]= evap[i]*np.minimum(1, SM[i-1]/(LP*FC)) # Actual Evapouration
        SM[i] = SM[i-1]+ INSOIL[i]-dUZ[i]-EA[i] # Soil water content


        #Upper Zone routine
        Q10[i]= np.minimum((UZ[i-1]+ dUZ[i]+Qdir[i]-np.minimum(PERC,UZ[i-1])), UZ1)*K1

        Q11[i]= np.maximum(0, (UZ[i-1]+ dUZ[i]+Qdir[i]-np.minimum(PERC,UZ[i-1]))-UZ1)*K2
        UZ[i] = UZ[i-1]+ dUZ[i]+Qdir[i]-np.minimum(PERC,UZ[i-1])- Q10[i]-Q11[i]

        #Lower zone routine
        QLZ[i] = KLZ*(LZ[i-1]+np.minimum(PERC,UZ[i-1]))+ LA*(prec[i]-evap[i])*KLZ
        LZ[i]= np.maximum(0, (LZ[i-1]+ np.minimum(PERC,UZ[i-1])- QLZ[i] + LA*(prec[i]-evap[i])))


        #simulated discharge

        Qsim[i]= (Q10[i]+Q11[i]+ QLZ[i])*(Area*10**3)/(86400)


    # Extracting coordinates 
    lat= ds['lat'].to_numpy()
    lon= ds['lon'].to_numpy()
    time= ds['time'].to_numpy()


    #Converting a numpy array to xarray
    Qsim_dr= xr.DataArray(Qsim,
                          coords={'time':time, 'lat':lat, 'lon':lon},
                         dims=['time','lat', 'lon'])

   
    Qsim_ds= Qsim_dr.to_dataset(name='Q', promote_attrs=True)
    
    basins_area = [
            ("subbasin1", shp1,3109),
            ("subbasin2", shp2,1630),
            ("subbasin3", shp3,297),
            ("subbasin4", shp4,2215),
            ]

    # Build a dict of DataFrames
    df_dict_r = {name: qsim_routed(clip2(Qsim_dr, shp),time,(area/450)) for name, shp, area,in basins_area}

    # Concatenate along columns;
    df_simq_r = pd.concat(df_dict_r, axis=1)

    # Flatten MultiIndex columns
    df_simq_r.columns = df_simq_r.columns.droplevel(1)
    
    return df_simq_r


In [ ]:
#initial guess for paramaters
     #Pcorr, Scorr, Tx, Ts, CX, FC, Beta, LP, K1, K2, UZ1, KLZ, PERC
initial_params= [1,1, -0.36, 1.15, 2.654,61.8, 4.5, 0.8, 0.022, 0.06, 82, 0.007, 0.65]


#Free parameters
#Pgrad, Tgrad, Area, CFR, CFRO, LA= params2
free_params= [0.65,0.5, 25, 0.01, 0.1, 0.138]


#Bounds for  model parameters
bounds =[(1,1), (1,1), (-1.5,1.5), (-1.5,1.5), (1.5,4.0), (50,100), (1,6),
        (0.75, 0.85), (0.01, 0.045), (0.045, 0.1), (40,100), (0.003, 0.01), (0.1,1)]

In [ ]:
#Run the model:


df_simq= hbv_model(params=initial_params,
                                params2=free_params,
                                prec= prec, 
                                temp= temp, 
                                evap= evap,
                                ds=ds)
df_simq.head()

### With MAXBIAS Routing

In [ ]:
#Run the model:
initial_params= [1,1, -0.36, 1.15, 2.654,61.8, 4.5, 0.8, 0.022, 0.06,82, 0.0085, 0.65]
df_simq_r= hbv_model_maxbas(params=initial_params,
                                params2=free_params,
                                prec= prec, 
                                temp= temp, 
                                evap= evap,
                                ds=ds)
df_simq_r.head()